In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, SimpleRNN, Dense, Dropout, Flatten, Concatenate, Reshape
from tensorflow.keras.backend import sin,cos, relu
from tensorflow.keras import Input
from tensorflow.keras.optimizers import SGD, Adam
from sklearn.model_selection import train_test_split

In [ ]:
dataset = pd.read_csv('../Data/train.csv')
train_size = int(dataset.shape[0] * 0.75)
df_train = dataset[:train_size]
df_test = dataset[train_size:]


def create_model():
    input1 = tf.keras.layers.Input(shape=(1,))
    x1 = tf.keras.layers.Dense(20, activation=tf.keras.backend.sin)(input1)
    x2 = tf.keras.layers.Dense(20, activation=tf.keras.backend.cos)(input1)
    x3 = tf.keras.layers.Dense(20, activation=tf.keras.backend.relu)(input1)
    added = tf.keras.layers.concatenate([x1, x2, x3])
    l0 = tf.keras.layers.Dropout(0.3)(added)
    l1 = tf.keras.layers.Dense(200, activation='relu')(l0)
    l2 = tf.keras.layers.Dropout(0.3)(l1)
    l3 = tf.keras.layers.Dense(200, activation='relu')(l2)
    out = tf.keras.layers.Dense(11, activation='softmax')(l3)
    model = tf.keras.models.Model(inputs=[input1], outputs=out)

    loss = tf.keras.losses.SparseCategoricalCrossentropy()
    # loss = tf.keras.losses.BinaryCrossentropy()
    # loss = tf.keras.losses.MeanSquaredError()
    opt = tf.keras.optimizers.Adam()
    # opt = tf.keras.optimizers.SGD
    basic_feature_extractor.compile(loss=loss, optimizer=opt, metrics=['accuracy'])

    return model


x = df_train.signal
y = df_train.open_channels
basic_feature_extractor = create_model()
# model.summary()
basic_feature_extractor.fit(x, y, epochs=1, validation_split=.3, verbose=1)

print('Done')
predictions = basic_feature_extractor.predict(x)

df_test = df_train.sample(frac=0.2)
x_test = df_test.signal
y_test = df_test.open_channels
prediction = basic_feature_extractor.evaluate(x_test, y_test)

print('Test Accuracy: %.3f' % prediction[1])


In [14]:
df_train = pd.read_csv('../Data/train.csv')
df_train['binary_open_channels'] = [int(x != 0) for x in df_train.open_channels]
open_channels = df_train.open_channels
df_train = pd.get_dummies(df_train, prefix = 'oc',columns=['open_channels'])
df_train['open_channels'] = open_channels
df_train

,time,signal,binary_open_channels,oc_0,oc_1,oc_2,oc_3,oc_4,oc_5,oc_6,oc_7,oc_8,oc_9,oc_10,open_channels
0,0.0001,-2.7600,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0.0002,-2.8557,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0.0003,-2.4074,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0.0004,-3.1404,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0.0005,-3.1525,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,499.9996,2.9194,1,0,0,0,0,0,0,0,1,0,0,0,7
4999996,499.9997,2.6980,1,0,0,0,0,0,0,0,1,0,0,0,7
4999997,499.9998,4.5164,1,0,0,0,0,0,0,0,0,1,0,0,8
4999998,499.9999,5.6397,1,0,0,0,0,0,0,0,0,0,1,0,9


In [21]:
signal = np.array(df_train.signal)
open_channels = np.array(df_train)
categorical_output = df_train[['oc_0','oc_1','oc_2','oc_3','oc_4','oc_5','oc_6','oc_7','oc_8','oc_9','oc_10']]

data = np.zeros((len(signal), 12))
data[:, 0] = signal
data[:, 1:] = categorical_output
data = data.reshape((1000, 5000, 12))
np.random.shuffle(data)

x = data[:,:, 0].reshape((1000,5000))
y = data[:,:, 1:].reshape((1000,5000,11))

In [22]:
np.shape(x),np.shape(y)

((1000, 5000), (1000, 5000, 11))

In [17]:
#multi_path_model_1 = tf.keras.models.load_model('../models/multi_path_model_60.h5')
multi_path_model_2 = tf.keras.models.load_model('../models/multi_path_model_71.h5')
flat_model = tf.keras.models.load_model('../models/resnet_flat_85.h5')
curvy_model = tf.keras.models.load_model('../models/resnet_curvy_49.h5')

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = .3)

In [34]:
def ensemble(sequence,verbose=False):
    second = flat_model.predict(sequence)
    third = curvy_model.predict(sequence)
    fourth = basic_feature_extractor.predict(sequence)
    sequence = sequence.reshape((1,5000,1))
    first = multi_path_model_2.predict(sequence)
    if verbose:
        print(first[0][0],second[0],third[0],fourth[0])
    results = [0]*5000
    for i in range(5000):
#         if np.amax(first) > 0.8:
#             results[i] = np.where(first == np.amax(first))
        result = first[0][i]#+fourth[i]
        results[i] = np.where(result == np.amax(result))
    return results

In [25]:
np.where(y_test[0][0] == 1), ensemble(x_test[0])[0], np.where(y_test[0][0] == 1) == ensemble(x_test[0])[0]

((array([3], dtype=int64),), (array([3], dtype=int64),), True)

In [35]:
correct = 0
incorrect = 0

for i in range(100):#range(len(x_test)):
    seq = x[i]
    pred = ensemble(seq)
    for j in range(5000):
        if pred[j] == np.where(y[i][j] == 1):
            correct += 1
        else:
            incorrect += 1

print('Accuracy:',correct/(correct+incorrect))

Accuracy: 0.70379
